In [ ]:
select * from tbl_yelp_reviews limit 10
select * from tbl_yelp_businesses limit 10

-- find number of business in each category
select * from tbl_yelp_businesses limit 10

with cte as (
select business_id, trim(A.value) as categories
from tbl_yelp_businesses
,LATERAL SPLIT_TO_TABLE(CATEGORIES, ',') AS A
)
select categories, count(*) as no_od_business
from cte 
group by 1
order by 2 desc

--2- Find the top 10 users who have reviewed the most businesses in the "Resturants" category.

select * from tbl_yelp_reviews limit 10
select * from tbl_yelp_businesses limit 10

select r.user_id, count(distinct r.business_id) as business_count
from tbl_yelp_reviews r
inner join tbl_yelp_businesses b 
on r.business_id = b.business_id
where b.categories ilike '%restaurant%'
group by user_id
order by business_count desc
limit 10 

--3- Find the most popular categories of businesses (based on the number of reviews).

select * from tbl_yelp_reviews limit 10
select * from tbl_yelp_businesses limit 10

with cte as (
select business_id, trim(A.value) as category
from tbl_yelp_businesses
,LATERAL SPLIT_TO_TABLE(CATEGORIES, ',') AS A
)
select category, count(*) as no_of_reviews
from cte 
inner join tbl_yelp_reviews r
on cte.business_id = r.business_id
group by 1
order by 2 desc

--4- Find the top 3 most recent reviews for each business

select * from tbl_yelp_reviews limit 10
select * from tbl_yelp_businesses limit 10

with cte as (
select r.*, b.name
,row_number() over(partition by b.business_id order by review_date) as rn
from tbl_yelp_reviews r
inner join tbl_yelp_businesses b
on r.business_id = b.business_id
)
select * from cte
where rn<=3

--5-Find the month with highest number of reviews.

select * from tbl_yelp_reviews limit 10
select * from tbl_yelp_businesses limit 10

select 
    month(review_date) as month_review,
    count(*) as no_of_review
from tbl_yelp_reviews
group by review_date
order by month_review desc;


--6-Find the percentage of 5-star reviews for each business

select * from tbl_yelp_reviews limit 10
select * from tbl_yelp_businesses limit 10

select 
    b.business_id,
    b.name,
    count(*) as total_review
,sum(case when review_stars = 5 then 1 else 0 end) as star5_reviews
,star5_reviews*100/total_review as percentile_5star
from tbl_yelp_reviews r
inner join tbl_yelp_businesses b
on r.business_id = b.business_id
group by b.business_id, b.name

--7-Find the top 5 most reviewd businesses in each city.

with cte as (
select 
    b.city,
    b.business_id,
    b.name,
    count(*) as total_review
from tbl_yelp_reviews r
inner join tbl_yelp_businesses b
on r.business_id = b.business_id
group by b.business_id, b.name, b.city
)
select *
from cte
qualify row_number() over(partition by city order by total_review desc) <=5

--8-Find the average rating of businesses that have at least 100 reviews.

select * from tbl_yelp_reviews limit 10
select * from tbl_yelp_businesses limit 10
 
select 
    b.business_id, 
    b.name,
    count(*) as total_review,
    avg(review_count) as avg_rating
from tbl_yelp_reviews r
inner join tbl_yelp_businesses b
on r.business_id=b.business_id
group by 
    b.business_id, 
    b.name
having count(*) >=100

--9-List the top 10 users who have written the most reviews, along with the businesses they reviewd.

with cte as (
select 
    r.user_id,
    count(*) as total_review
from tbl_yelp_reviews r
inner join tbl_yelp_businesses b
on r.business_id = b.business_id
group by r.user_id
order by total_review desc
limit 10
)
select user_id, business_id
from tbl_yelp_reviews
where user_id in (select user_id from cte)
group by user_id, business_id
order by user_id

--10- Find top 10 businesses with highest positive sentiment review

select 
    r.business_id,
    b.name,
    --r.user_id,
    count(*) as total_review
from tbl_yelp_reviews r
inner join tbl_yelp_businesses b
on r.business_id = b.business_id
where r.sentiments = 'Positive'
group by r.business_id, b.name
order by total_review desc
limit 10
